<a href="https://colab.research.google.com/github/somilasthana/MachineLearningSkills/blob/master/PySpark_Basic_DataFrame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
!tar xf spark-2.3.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"


In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [26]:
!curl -L -o donation.zip https://bit.ly/1Aoywaq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   163  100   163    0     0    507      0 --:--:-- --:--:-- --:--:--   507
100 53.8M  100 53.8M    0     0  44.4M      0  0:00:01  0:00:01 --:--:-- 65.5M


In [0]:
!mkdir linkage
!cp /content/donation.zip /content/linkage/

In [31]:
%cd /content/linkage/

/content/linkage


In [32]:
!unzip /content/linkage/donation.zip

Archive:  /content/linkage/donation.zip
 extracting: block_10.zip            
 extracting: block_1.zip             
 extracting: block_2.zip             
 extracting: block_3.zip             
 extracting: block_4.zip             
 extracting: block_5.zip             
 extracting: block_6.zip             
 extracting: block_7.zip             
 extracting: block_8.zip             
 extracting: block_9.zip             
  inflating: documentation           
  inflating: frequencies.csv         


In [34]:
!ls

block_10.zip  block_3.zip  block_6.zip	block_9.zip    frequencies.csv
block_1.zip   block_4.zip  block_7.zip	documentation
block_2.zip   block_5.zip  block_8.zip	donation.zip


In [36]:
!unzip /content/linkage/block_1.zip
!unzip /content/linkage/block_10.zip
!unzip /content/linkage/block_2.zip
!unzip /content/linkage/block_3.zip
!unzip /content/linkage/block_4.zip
!unzip /content/linkage/block_5.zip
!unzip /content/linkage/block_6.zip
!unzip /content/linkage/block_7.zip
!unzip /content/linkage/block_8.zip
!unzip /content/linkage/block_9.zip

Archive:  /content/linkage/block_1.zip
  inflating: block_1.csv             
Archive:  /content/linkage/block_10.zip
  inflating: block_10.csv            
Archive:  /content/linkage/block_2.zip
  inflating: block_2.csv             
Archive:  /content/linkage/block_3.zip
  inflating: block_3.csv             
Archive:  /content/linkage/block_4.zip
  inflating: block_4.csv             
Archive:  /content/linkage/block_5.zip
  inflating: block_5.csv             
Archive:  /content/linkage/block_6.zip
  inflating: block_6.csv             
Archive:  /content/linkage/block_7.zip
  inflating: block_7.csv             
Archive:  /content/linkage/block_8.zip
  inflating: block_8.csv             
Archive:  /content/linkage/block_9.zip
  inflating: block_9.csv             


In [0]:
!rm -rf /content/linkage/*.zip

In [0]:
prev = spark.read.option("header", "true").option("nullValue", "?").option("inferSchema", "true").csv("/content/linkage")

In [46]:
prev.show()

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|    true|
|25739|45991|         1.0|        null|         1.0|        null|      1|     1|     1|     1|      1|  

In [47]:
prev.printSchema()

root
 |-- id_1: integer (nullable = true)
 |-- id_2: integer (nullable = true)
 |-- cmp_fname_c1: double (nullable = true)
 |-- cmp_fname_c2: double (nullable = true)
 |-- cmp_lname_c1: double (nullable = true)
 |-- cmp_lname_c2: double (nullable = true)
 |-- cmp_sex: integer (nullable = true)
 |-- cmp_bd: integer (nullable = true)
 |-- cmp_bm: integer (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [43]:
prev.count()

5749142

In [50]:
prev.groupBy("is_match").count().orderBy("count").show()

+--------+-------+
|is_match|  count|
+--------+-------+
|    true|  20931|
|   false|5728201|
+--------+-------+



In [54]:
import pyspark.sql.functions
prev.agg(pyspark.sql.functions.avg("cmp_sex"), pyspark.sql.functions.stddev("cmp_sex")).show()

+-----------------+--------------------+
|     avg(cmp_sex)|stddev_samp(cmp_sex)|
+-----------------+--------------------+
|0.955001381078048| 0.20730111116897443|
+-----------------+--------------------+



In [0]:
prev.createOrReplaceTempView("linkage")

In [56]:
spark.sql("""
SELECT is_match, COUNT(*) cnt FROM linkage GROUP BY is_match ORDER BY cnt DESC
""").show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
+--------+-------+



In [0]:
summary = prev.describe()

In [58]:
summary.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|summary|              id_1|              id_2|      cmp_fname_c1|      cmp_fname_c2|       cmp_lname_c1|       cmp_lname_c2|            cmp_sex|             cmp_bd|             cmp_bm|            cmp_by|            cmp_plz|
+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+-------------------+
|  count|           5749132|           5749132|           5748125|            103698|            5749132|               2464|            5749132|            5748337|            5748337|           5748337|            5736289|
|   mean| 33324.48559643438| 66587.43558331935|0.7129024704425707| 0.900017671890335|0.3156278193076

In [59]:
summary.select("cmp_fname_c1", "cmp_fname_c2").show()

+------------------+------------------+
|      cmp_fname_c1|      cmp_fname_c2|
+------------------+------------------+
|           5748125|            103698|
|0.7129024704425707| 0.900017671890335|
|0.3887583596162788|0.2713176105782331|
|               0.0|               0.0|
|               1.0|               1.0|
+------------------+------------------+



In [0]:
matches = prev.where("is_match == true")
matchSummary =matches.describe()

In [0]:
matches = prev.where("is_match == false")
missSummary =matches.describe()

In [153]:
missSummary.show()

+-------+------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+--------------------+
|summary|              id_1|             id_2|      cmp_fname_c1|      cmp_fname_c2|      cmp_lname_c1|       cmp_lname_c2|            cmp_sex|            cmp_bd|            cmp_bm|             cmp_by|             cmp_plz|
+-------+------------------+-----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+--------------------+
|  count|           5728201|          5728201|           5727203|            102365|           5728201|               1989|            5728201|           5727412|           5727412|            5727412|             5715387|
|   mean|33319.913548075565|66643.44259218557|0.7118634802163704| 0.898847351409032|0.3131380113360652|0.162

In [62]:
matchSummary.show()

+-------+------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------------------+
|summary|              id_1|              id_2|       cmp_fname_c1|       cmp_fname_c2|       cmp_lname_c1|      cmp_lname_c2|            cmp_sex|             cmp_bd|              cmp_bm|             cmp_by|           cmp_plz|
+-------+------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+--------------------+-------------------+------------------+
|  count|             20931|             20931|              20922|               1333|              20931|               475|              20931|              20925|               20925|              20925|             20902|
|   mean| 34575.72117911232| 51259.95939037791| 0.9973163859635039| 0.9898900320318176| 0.99

In [0]:
schema = summary.schema

In [0]:
def fn(x):
  metric = x[0]
  final = []
  for i,val in enumerate(x[1:]):
    result = [metric]
    result.append(schema[i+1].name)
    result.append(float(val))
    final.append(result)
    
  return final
  
longForm = summary.rdd.flatMap(fn)

In [139]:
longForm.take(3)

[['count', 'id_1', 5749132.0],
 ['count', 'id_2', 5749132.0],
 ['count', 'cmp_fname_c1', 5748125.0]]

In [0]:
longFormdf = longForm.map(lambda x: (x[0], x[1], x[2])).toDF(("metric", "name", "value"))

In [142]:
longFormdf.show()

+------+------------+-------------------+
|metric|        name|              value|
+------+------------+-------------------+
| count|        id_1|          5749132.0|
| count|        id_2|          5749132.0|
| count|cmp_fname_c1|          5748125.0|
| count|cmp_fname_c2|           103698.0|
| count|cmp_lname_c1|          5749132.0|
| count|cmp_lname_c2|             2464.0|
| count|     cmp_sex|          5749132.0|
| count|      cmp_bd|          5748337.0|
| count|      cmp_bm|          5748337.0|
| count|      cmp_by|          5748337.0|
| count|     cmp_plz|          5736289.0|
|  mean|        id_1|  33324.48559643438|
|  mean|        id_2|  66587.43558331935|
|  mean|cmp_fname_c1| 0.7129024704425707|
|  mean|cmp_fname_c2|  0.900017671890335|
|  mean|cmp_lname_c1|0.31562781930763056|
|  mean|cmp_lname_c2|0.31841283153174366|
|  mean|     cmp_sex|  0.955001381078048|
|  mean|      cmp_bd|0.22446526708507172|
|  mean|      cmp_bm|0.48885529849763504|
+------+------------+-------------

In [0]:
wideFormdf = longFormdf.groupBy("name").pivot("metric", ["count", "mean", "stddev", "min", "max"]).agg(pyspark.sql.functions.first("value"))

In [148]:
wideFormdf.show()

+------------+---------+-------------------+-------------------+---+--------+
|        name|    count|               mean|             stddev|min|     max|
+------------+---------+-------------------+-------------------+---+--------+
|        id_2|5749132.0|  66587.43558331935| 23620.487613269706|6.0|100000.0|
|     cmp_plz|5736289.0|0.00552866147434343|0.07414914925420013|0.0|     1.0|
|cmp_lname_c1|5749132.0|0.31562781930763056| 0.3342336339615803|0.0|     1.0|
|cmp_lname_c2|   2464.0|0.31841283153174366| 0.3685670662006655|0.0|     1.0|
|     cmp_sex|5749132.0|  0.955001381078048|0.20730111116897443|0.0|     1.0|
|      cmp_bm|5748337.0|0.48885529849763504| 0.4998758236779003|0.0|     1.0|
|cmp_fname_c2| 103698.0|  0.900017671890335| 0.2713176105782331|0.0|     1.0|
|cmp_fname_c1|5748125.0| 0.7129024704425707| 0.3887583596162788|0.0|     1.0|
|        id_1|5749132.0|  33324.48559643438| 23659.859374487987|1.0| 99980.0|
|      cmp_bd|5748337.0|0.22446526708507172|0.41722972238461925|

In [0]:
matchSummary.createOrReplaceTempView("match_desc")
missSummary.createOrReplaceTempView("miss_desc")